In [20]:
from transformers import pipeline
import torch
import pandas as pd
from tqdm import tqdm
import nltk
import numpy as np

In [ ]:
# Set the device to MPS if available

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load the pipeline and move the model to MPS
generator = pipeline(
    "text-generation",
    model="KennethTM/gpt-neo-1.3B-danish",
    device= "mps" if torch.backends.mps.is_available() else "cpu"

)

Using device: mps


In [5]:
qa = pd.read_excel('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/Legal questions Adam.xlsx')
qa.head()

,"question, str","answer, str","text, str","pnumber, str","law number, str"
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør",'Ejeren af en ejerlejlighed har sammen med and...,3,LOV nr 908 af 18/06/2020
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,'Højskolen skal følge de af finansministeren f...,30,LBK nr 780 af 08/08/2019
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,'Uden ugrundet ophold efter repræsentantskabet...,25 l,LBK nr 1110 af 10/10/2014
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,'Følgende grænser for Arbejdsmarkedets Tillægs...,26 e,LBK nr 1110 af 10/10/2014
4,Hvad er en betingelse for retten til jobpræmie?,Det er en betingelse for retten til jobpræmie ...,'Det er en betingelse for retten til jobpræmie...,9,LOV nr 287 af 29/03/2017


In [9]:
questions = [item for item in qa['question, str']]

answers = []

for item in tqdm(questions, desc='Answering questions'):

    # Perform text generation
    text = generator(
        f"Spørgsmål: {item} Svar: ",
        max_length=100,
        num_return_sequences=1,  # Generate a single sequence
        do_sample=True
    )

    answers.append(text[0]["generated_text"])

Answering questions: 100%|██████████| 39/39 [06:16<00:00,  9.66s/it]


In [14]:
type(answers[0])

str

In [15]:
answers2 = [item.split('Svar:')[1] for item in answers]

In [16]:
answers2

[' \x03Det er ejerlejlighedernes ejere, der har ret til at anvende fællesarealer, herunder fælles vaskeri, på egen foranledning. Dette skal ske efter regler for fordeling af beboernes brugsret. Hvad er fordele og ulemper? ',
 ' ʺStk. 1. Løn- og ansættelsesvilkår fastsættes af bestyrelsen i samarbejde med skolens ansatte i overensstemmelse med den til enhver tid gældende overenskomst mellem finansministeren og en faglig organisation inden for det pågældende område. Stk. 2. Bestyrelsen for skolen har ansvaret for, at løn- og ansættelsesvilkår til skolens ansatte til enhver tid afspejler de krav om løn- og ansættelsesvilkår, der er beskrevet i aftalen',
 ' ʺFor at en redegørelse (rapportering, redegørelse for den finansielle stilling, redegørelse for virksomhedsledelse eller ledelsesberetning) kan betragtes som fyldestgørende og dermed afgives til tilsynsmyndigheden, skal ledelsesberetning for det seneste regnskabsår være suppleret med redegørelse for den finansielle stilling og ledelsesb

In [18]:
# calculate BLEU-scores for this:

bleu_scores = []
idx = 0

for pred_answer, true_answer in zip(answers, list(qa['answer, str'])):
    try:
        bleu_scores.append(nltk.translate.bleu_score.sentence_bleu([true_answer], pred_answer))
    except:
        print(f'Error at index {idx}')
    idx += 1

Error at index 35


/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [21]:
np.mean(bleu_scores)

0.10743160673601584